In [1]:
from fastapi import FastAPI, HTTPException
from typing import List
import spacy

# Load the Macedonian language model
nlp = spacy.load("mk_core_news_sm")

app = FastAPI()

@app.post("/filter_sentences/")
async def filter_sentences(sentences: List[str], keyword: str, pos_category: str):
    matching_sentences = []
    for sentence in sentences:
        doc = nlp(sentence)
        for token in doc:
            if token.text.lower() == keyword.lower() and pos_category.lower() in spacy.explain(token.pos_).lower():
                matching_sentences.append(sentence)
                break
    return {"matching_sentences": matching_sentences}

In [7]:
def pos_tag_text(text, keyword, pos_category):
    results = []
    doc = nlp(text)  # Process the whole text

    for token in doc:  # Iterate over tokens in the document
        # Check if the token matches the keyword and the specified part of speech category
        if token.text.lower() == keyword.lower() and pos_category.lower() in spacy.explain(token.pos_).lower():
            # Define the context window or simply split the text around the keyword
            start_index = token.idx  # Start index of the keyword in the text
            end_index = start_index + len(token.text)  # End index of the keyword in the text

            # Extracting left and right context
            left_context = text[max(0, start_index - 30):start_index].strip()  # 30 characters as an example window size
            right_context = text[end_index:min(len(text), end_index + 30)].strip()  # Adjust context window as needed

            results.append((left_context, token.text, right_context))
    
    return {"occurrences": results}

In [3]:
sentences = [
        "Петар одлучи да се врати во Австралија за 2 недели.",
        "Втора реченица за пример.",
        "Ова е исто така трет документ.",
        "Ова е тест датотека."
    ]

In [8]:
text = '''Патникот што му даде веленце на другарот.



Си патувал еден патник од еден град во друг и се здружил со некој друг патник. Одејќи си двајцата така, се задал еден силен облак со дожд и гледајќи дека ќе врне многу дожд, му побарал едниот на другиот патник едно веленце (прекривка) да се наметнe, за да не го натопи дождот. Одејќи до некаде дождот престанал и патниците останале суви.

— Ама ако не ти го дадев веленцето, пријателе, жива вода ќе се стореше — му рекол.

— Ти благодарам пријателе што ми го даде веленцето и не наврнав. Многу ти благодарам! — рекол патникот.

Пооди до некаде и пак ќе му речел.

— А, пријателе? Да не ти го дадев веленцето, жива вода ќе се стореше.

Пооди до негде, пак истото.

Одејќи така дошле до една река и слегле да се одморат под сенките, а таму имало и други што се одморале. Во часот кога седнал меѓу луѓето, патникот почнал да се фали дека на другарот му го дал веленцето да не го наврне дождот. Речи, па речи, речи, па речи, па најпосле другар му се разлутил, станал од кај што седел па се фрлил во реката и се направил жива вода.

— Е доста ме мачи бре чоече, со тоа твое палено веленце: „Веленцето ако не ти го дадев, дождот ќе те истопеше, веленцето ако не ти го дадев жива вода ќе станеше!“ Е, од олку повеќе ќе се истопев? Бре нечоек! На’, еве се истопив повеќе од што дождот ќе ме истопеше, само и само да те замолчам, да не ми се фалиш, оти ме пукна со тоа твое веленце што ми го даде! — му рекол на лутина, си го јавнал коњот, па си отишол по патот сам.'''

In [9]:
ms = pos_tag_text(text, keyword='вода', pos_category='noun')

In [10]:
ms

{'occurrences': [('ев веленцето, пријателе, жива',
   'вода',
   'ќе се стореше — му рекол.\n\n—'),
  ('е ти го дадев веленцето, жива', 'вода', 'ќе се стореше.\n\nПооди до негд'),
  ('во реката и се направил жива', 'вода', '.\n\n— Е доста ме мачи бре чоече'),
  ('нцето ако не ти го дадев жива', 'вода', 'ќе станеше!“ Е, од олку повеќ')]}